## Assignment 4
### Gagana Uday Kumar (WOV796)

### 3. We now review k-fold cross-validation.

### (a) Explain how k-fold cross-validation is implemented.

K-fold cross-validation involves splitting the dataset into k subsets. The model is trained k times, each time using a different subset as the validation set and the remaining data as the training set. Performance metrics are calculated for each iteration, and the average performance across all iterations is used to evaluate the model's performance. This method helps assess the model's generalization ability and reduces the risk of overfitting or underfitting

### (b) What are the advantages and disadvantages of k-fold crossvalidation relative to:

### i. The validation set approach?

K-fold cross-validation offers several advantages over the validation set approach. Firstly, it maximizes the utilization of available data by partitioning the dataset into k equal-sized folds, ensuring that each data point is used for both training and validation across different folds. This leads to a more robust estimation of model performance, as it reduces the risk of overfitting to a specific validation set. Additionally, k-fold cross-validation provides a more stable estimate of model performance by averaging the results obtained from multiple rounds of training and validation with different subsets of data. This mitigates the variability associated with using a single validation set and provides a more reliable assessment of the model's generalization ability.

However, k-fold cross-validation also comes with some drawbacks compared to the validation set approach. Firstly, it entails a higher computational cost since the model needs to be trained and evaluated k times, which can be resource-intensive for large datasets or complex models. This increased computational burden may not be feasible in scenarios where computational resources are limited. Moreover, the training time escalates with the number of folds, further exacerbating the computational overhead. Secondly, implementing k-fold cross-validation requires additional programming and handling of multiple folds, introducing complexity to the code compared to the simpler validation set approach. This complexity may pose challenges, particularly for beginners or when integrating cross-validation into existing workflows.

### ii. LOOCV?

K-fold cross-validation (KCV) and Leave-One-Out Cross-Validation (LOOCV) are two commonly used techniques for estimating the performance of machine learning models. KCV divides the dataset into k subsets or folds, trains the model k times using k-1 folds for training and one fold for validation, and then averages the results. In contrast, LOOCV partitions the dataset into n subsets, where n is the number of instances in the dataset, and trains the model n times, each time leaving out one instance for validation.

One advantage of KCV over LOOCV is its reduced computational complexity. KCV typically requires fewer iterations compared to LOOCV, as the number of folds (k) is usually much smaller than the number of instances (n). This results in faster model evaluation, making KCV more practical for larger datasets. Additionally, KCV tends to provide a more stable estimate of model performance than LOOCV. Since KCV averages the results over multiple iterations with different subsets of data, it reduces the variance in the performance estimate and provides a more reliable assessment of the model's generalization ability.

However, KCV also has some disadvantages compared to LOOCV. One limitation is that KCV may introduce higher bias into the performance estimate, especially when the dataset is small. With a smaller number of folds, each training set may not be representative of the entire dataset, leading to a biased evaluation of the model. Additionally, the choice of the number of folds (k) in KCV can impact the performance estimate. Selecting an inappropriate value for k may result in either overfitting or underfitting of the model to the data, affecting the reliability of the performance estimate. In contrast, LOOCV does not suffer from bias introduced by the choice of folds and provides an unbiased estimate of model performance, albeit at a higher computational cost.

### 5. In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the Default data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from ISLP import load_data
from ISLP.models import ModelSpec as MS, summarize
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
(LinearDiscriminantAnalysis as LDA ,
QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB


In [3]:
De = pd.read_csv('Default.csv',na_values=['?']).dropna()
De

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


In [4]:
De.describe()

,balance,income
count,10000.000000,10000.000000
mean,835.374886,33516.981876
std,483.714985,13336.639563
min,0.000000,771.967729
25%,481.731105,21340.462903
50%,823.636973,34552.644802
75%,1166.308386,43807.729272
max,2654.322576,73554.233495


In [6]:
De.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  object 
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB


In [8]:
De["default"].value_counts()

No     9667
Yes     333
Name: default, dtype: int64

In [15]:
De['default'].replace({'Yes': 1, 'No': 0}, inplace=True)
De['default'] = pd.to_numeric(De['default'], errors='coerce')

In [16]:
De.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  int64  
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 312.6+ KB


### (a) Fit a logistic regression model that uses income and balance to predict default.

In [23]:
np.random.seed(42)

In [24]:
allvars = De.columns.drop(['default', 'student',])
design = MS(allvars)
X = design.fit_transform(De)
y = De.default == 1
glm = sm.GLM(y,X, family=sm.families.Binomial())
results = glm.fit()
summarize(results)

,coef,std err,z,P>|z|
intercept,-11.540500,0.435000,-26.544,0.0
balance,0.005600,0.000000,24.835,0.0
income,0.000021,0.000005,4.174,0.0


In [25]:
X = De[['income', 'balance']]
y = De['default']
model = LogisticRegression()
model.fit(X, y)
print("Coefficients:", model.coef_)

Coefficients: [[2.08091984e-05 5.64710797e-03]]


### (b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

### i. Split the sample set into a training set and a validation set.

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### ii. Fit a multiple logistic regression model using only the training observations.

In [27]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

### iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.

In [28]:
y_pred = model.predict(X_val)

### iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [32]:
validation_error = 1 - accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Validation Set Error:", validation_error)


Accuracy: 0.9655
Validation Set Error: 0.034499999999999975


### (c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [38]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

validation_error = 1 - accuracy_score(y_val, y_pred)
print("Accuracy for test size 0.25:", accuracy_score(y_val, y_pred))
print("Validation Set Error for test size 0.25:", validation_error)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

validation_error = 1 - accuracy_score(y_val, y_pred)
print("Accuracy for test size 0.30:", accuracy_score(y_val, y_pred))
print("Validation Set Error for test size 0.30:", validation_error)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.35, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

validation_error = 1 - accuracy_score(y_val, y_pred)
print("Accuracy for test size 0.35:", accuracy_score(y_val, y_pred))
print("Validation Set Error for test size 0.35:", validation_error)

Accuracy for test size 0.25: 0.9724
Validation Set Error for test size 0.25: 0.027599999999999958
Accuracy for test size 0.30: 0.9683333333333334
Validation Set Error for test size 0.30: 0.03166666666666662
Accuracy for test size 0.35: 0.9731428571428572
Validation Set Error for test size 0.35: 0.0268571428571428


### (d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [103]:
X = pd.concat([De[['income', 'balance']], pd.get_dummies(De['student'], drop_first=True)], axis=1)
y = De['default']

validation_errors = []


# Step i: Split the sample set into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=i*10)  # Change random_state value
    
# Step ii: Fit a multiple logistic regression model using only the training observations
model = LogisticRegression()
model.fit(X_train, y_train)
    
# Step iii: Obtain predictions for the validation set
y_pred = model.predict(X_val)
    
# Step iv: Compute the validation set error
validation_error = 1 - accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Validation Set Error: ", validation_error)

Accuracy: 0.9695
Validation Set Error:  0.03049999999999997


Including student in the model does not improve the test erro rate much but we can see very little difference in the test error rate

### 6. We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the sm.GLM() function. Do not forget to set a random seed before beginning your analysis.

### (a) Using the summarize() and sm.GLM() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors.

In [81]:
glm = sm.GLM(y,X, family=sm.families.Binomial())
results = glm.fit()

print("Standard Errors:")
print("Income:", standard_errors['income'])
print("Balance:", standard_errors['balance'])
summarize(results)

Standard Errors:
Income: 4.985245458257281e-06
Balance: 0.0002273813844021027


,coef,std err,z,P>|z|
income,-0.0002,0.000005,-33.672,0.0
balance,0.0028,0.000000,22.291,0.0
Yes,-3.7531,0.153000,-24.580,0.0


### (b) Write a function, boot_fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model.

In [106]:
def boot_fn(data, index):
    # Extract subset of data based on index
    subset_data = data.iloc[index]
    
    # Fit logistic regression model using subset of data
    logit_model = sm.GLM.from_formula('default ~ income + balance', data=subset_data, family=sm.families.Binomial()).fit()
    
    # Return coefficient estimates for income and balance
    return logit_model.params['income'], logit_model.params['balance']


### (c) Following the bootstrap example in the lab, use your boot_fn() function to estimate the standard errors of the logistic regression coefficients for income and balance.

In [86]:
def bootstrap(data, num_iterations):
    num_obs = len(data)
    standard_errors_income = []
    standard_errors_balance = []
    
    # Iterate through bootstrap iterations
    for _ in range(num_iterations):
        # Generate bootstrap sample indices
        bootstrap_indices = np.random.choice(num_obs, size=num_obs, replace=True)
        
        # Calculate standard errors using boot_fn
        se_income, se_balance = boot_fn(data, bootstrap_indices)
        standard_errors_income.append(se_income)
        standard_errors_balance.append(se_balance)
    
    # Calculate average standard errors
    avg_se_income = np.mean(standard_errors_income)
    avg_se_balance = np.mean(standard_errors_balance)
    
    return avg_se_income, avg_se_balance

# Call bootstrap function
standard_errors_income, standard_errors_balance = bootstrap(De, num_iterations=1000)

# Print standard errors
print("Standard Errors:")
print("Income:", standard_errors_income)
print("Balance:", standard_errors_balance)

Standard Errors:
Income: 2.0869252932309845e-05
Balance: 0.005653063123311437


### (d) Comment on the estimated standard errors obtained using the sm.GLM() function and using the bootstrap.

We can observe that the standard errors obtained using the bootstrap are generally larger compared to those obtained directly from the sm.GLM() function. This difference is expected because the bootstrap method is inherently more robust and accounts for the variability in the data by resampling. Therefore, it tends to provide more conservative estimates of the standard errors.

In this case, the larger standard errors obtained from the bootstrap indicate that there is more uncertainty associated with the coefficient estimates for both income and balance when using the bootstrap method. This suggests that the original standard errors obtained from the sm.GLM() function may have underestimated the uncertainty in the coefficient estimates.

### 9. We will now consider the Boston housing data set, from the ISLP library.

In [87]:
Boston = pd.read_csv('Boston.csv',na_values=['?']).dropna()
Boston

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,502,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,9.67,22.4
502,503,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,9.08,20.6
503,504,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,5.64,23.9
504,505,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,6.48,22.0


In [88]:
Boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  506 non-null    int64  
 1   crim        506 non-null    float64
 2   zn          506 non-null    float64
 3   indus       506 non-null    float64
 4   chas        506 non-null    int64  
 5   nox         506 non-null    float64
 6   rm          506 non-null    float64
 7   age         506 non-null    float64
 8   dis         506 non-null    float64
 9   rad         506 non-null    int64  
 10  tax         506 non-null    int64  
 11  ptratio     506 non-null    float64
 12  lstat       506 non-null    float64
 13  medv        506 non-null    float64
dtypes: float64(10), int64(4)
memory usage: 55.5 KB


In [89]:
Boston.describe()

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,253.500000,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806
std,146.213884,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104
min,1.000000,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000
25%,127.250000,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000
50%,253.500000,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000
75%,379.750000,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000
max,506.000000,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000


### (a) Based on this data set, provide an estimate for the population mean of medv. Call this estimate $\hat{μ}$.

In [90]:
sample_mean = Boston['medv'].mean()

# Print the estimated population mean
print("Estimated population mean of medv (μ̂):", sample_mean)

Estimated population mean of medv (μ̂): 22.532806324110677


### (b) Provide an estimate of the standard error of $\hat{μ}$. Interpret this result. Hint: We can compute the standard error of the sample mean by dividing the sample standard deviation by the square root of the number of observations.

In [91]:
# Calculate sample standard deviation
sample_std = Boston['medv'].std()

# Determine the number of observations
n = Boston['medv'].count()

# Calculate the standard error of the sample mean
SE = sample_std / np.sqrt(n)

# Print the estimate of the standard error
print("Estimated standard error of the sample mean (SE):", SE)

Estimated standard error of the sample mean (SE): 0.4088611474975351


### (c) Now estimate the standard error of $\hat{μ}$ using the bootstrap. How does this compare to your answer from (b)?

In [94]:
from sklearn.utils import resample
def boot(df):
    return resample(df)

B = 1000
sample_mean = []

for i in range(B):
    df = boot(Boston)
    sample_mean.append(df['medv'].mean())

print("Estimate of standard error of sample mean of medv (using bootstrap) is: " +str(np.std(sample_mean, ddof=1)))

Estimate of standard error of sample mean of medv (using bootstrap) is: 0.40420840484829434


### (d) Based on your bootstrap estimate from (c), provide a 95 % confidence interval for the mean of medv. Compare it to the results obtained by using Boston['medv'].std() and the two standard error rule (3.9). Hint: You can approximate a 95 % confidence interval using the formula [$\hat{μ}$ − 2SE($\hat{μ}$), $\hat{μ}$ + 2SE($\hat{μ}$)].

In [96]:
import scipy.stats as stats

print("95% Confidence interval using bootstrap is: (" + str(22.5328 - 0.40932) + "," + str(22.5328 + 0.40932) + ")")
stats.ttest_1samp(a= Boston['medv'], popmean=0)

95% Confidence interval using bootstrap is: (22.12348,22.942120000000003)


TtestResult(statistic=55.11114583037392, pvalue=9.370623727132662e-216, df=505)

### (e) Based on this data set, provide an estimate, $\hat{μ}_{med}$, for the median value of medv in the population.

In [98]:
print("Estimate for population median of medv is: " +str(Boston['medv'].median()))

Estimate for population median of medv is: 21.2


### (f) We now would like to estimate the standard error of $\hat{μ}_{med}$. Unfortunately, there is no simple formula for computing the standard error of the median. Instead, estimate the standard error of the median using the bootstrap. Comment on your findings.

In [99]:
B = 1000
sample_median = []

for i in range(B):
    df = boot(Boston)
    sample_median.append(df['medv'].median())

print("Estimate of standard error of sample median of medv (using bootstrap) is: " +str(np.std(sample_median, ddof=1)))

Estimate of standard error of sample median of medv (using bootstrap) is: 0.3787820657757612


### (g) Based on this data set, provide an estimate for the tenth percentile of medv in Boston census tracts. Call this quantity $\hat{μ}_{0.1}$. (You can use the np.percentile() function.)

In [101]:
print("Estimate for the tenth percentile of medv is: " +str(Boston['medv'].quantile(q=0.1)))


Estimate for the tenth percentile of medv is: 12.75


### (h) Use the bootstrap to estimate the standard error of $\hat{μ}_{0.1}$ . Comment on your findings.

In [102]:
B = 1000
sample_percentile = []

for i in range(B):
    df = boot(Boston)
    sample_percentile.append(df['medv'].quantile(q=0.1))

print("Estimate of standard error for the tenth percentile of medv (using bootstrap) is: "
      +str(np.std(sample_percentile, ddof=1)))

Estimate of standard error for the tenth percentile of medv (using bootstrap) is: 0.5070456935379052
